# Chương 3: Triển khai cơ chế Attention (Attention Mechanisms)

In [5]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.7.0


<img src="https://images.viblo.asia/634f6787-7c44-4939-b256-35a7bc476a38.png" width="800px" >

Các phương pháp **attention** sẽ tìm hiểu ở phần này.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

1. Self-attention đơn giản: Loại đơn giản nhất với các trọng số cố định => không huấn luyện được
2. Self-attention phiên bản có trọng số có thể huấn luyện được (trainable weights)
3. Causal attention (hay masked attention): Đảm bảo mô hình chỉ "nhìn thấy" các **token** trước đó trong văn bản.
4. Multi-head attention: Phương pháp được ứng dựng thực tiễn trong các mô hình ngôn ngữ lớn, gồm nhiều bước xử lý song song.


## 3.1 Vấn đề của mô hình khi xử lý một chuỗi văn bản dài

- Nếu dịch từng từ một theo thứ tự như ảnh dưới đây thì chắc chắn câu văn dịch ra sẽ rất rời rạc, khó hiểu.
- Do ngữ pháp của các ngôn ngữ khác nhau, thứ tự các từ sau khi dịch có thể khác nhiều so với câu gốc.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="800px">

Trước khi kiến trúc **Transformer** ra đời, **RNN (Recurrent Neural Networks)** là phương pháp phổ biến nhất áp dụng cho các tác vụ dịch thuật. 

**RNN** là một mạng nơ-ron trong đó đầu ra từ các bước trước được đưa vào làm đầu vào cho bước hiện tại, khiến chúng rất phù hợp với dữ liệu tuần tự như văn bản. Trạng thái tại mỗi bước xử lý của RNN được gọi là **hidden state**.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="800px">

Tuy nhiên, RNN thường gặp khó khăn với các chuỗi dài, khi chúng có thể bị "quên" đi các trạng thái cách xa trước đó. Các xử lý tuần tự cũng khiến cho RNN chậm hơn so với các phương pháp xử lý song song như **Transformer**.

## 3.2 Attention

Năm 2014, một nghiên cứu có tên **Bahdanau attention** được công bố, với nội dung chính là đề xuất một phương pháp chỉnh sửa **encoder-decoder** của RNN sao cho **decoder** có thể truy cập được vào tất cả các *token* của văn bản đầu vào. 

Dựa vào trọng số (attention weight) được tính toán, mô hình sẽ chọn ra đâu là từ tiếp theo phù hợp để dịch và đưa vào kết quả.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="800px">

Hình trên mô tả bước thứ 2 trong tác vụ dịch, trọng số lớn nhất mà mô hình tính toán được ở bước này nằm ở từ **du** => Từ tiếp theo được trả về sẽ lẽ **you** (từ tương ứng của **du** trong tiếng Anh)

> **Self-attention** là cơ chế giúp mô hình hiểu mối quan hệ giữa các từ trong cùng một câu, bất kể khoảng cách giữa chúng. Đây là thành phần cốt lõi của Transformer, LLMs giúp nó vượt trội so với RNN.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="800px">

## 3.3 Các kiểu cơ chế Attention

### 3.3.1 Self-Attention đơn giản (simple self-attention)

Trọng số attention trong phương pháp này được tính trực tiếp từ dữ liệu đầu vào. Cùng xem ví dụ minh họa dưới đây.

<img src="https://images.viblo.asia/29375700-b7f7-4320-858b-3139406a5e08.png" width="800px">

- Trên hình là ví dụ trong việc tính mối quan hệ giữa từ `journey` với các từ còn lại trong câu.
- Để tính được vector `z` từ các  **token embeddings** sẽ có các **trọng số attention** `α`.
- Để có được `α`, lại cần phải tính toán giá trị **attention score** (`ω`) trước.

### Tính Attention score

Để tính được các giá trị **attention score** với từ `journey`, ta lấy vector $x^{(2)}$ nhân vô hướng với các vector còn lại. vector  $x^{(2)}$ trong trường hợp này sẽ được gọi với cái tên **embedded query token**.

$\omega_{2i} = x^{(2)} \times x^{(i)}$

In [6]:
import torch

inputs = torch.tensor(
 [[0.43, 0.15, 0.89], # Your (x^1)
 [0.55, 0.87, 0.66], # journey (x^2)
 [0.57, 0.85, 0.64], # starts (x^3)
 [0.22, 0.58, 0.33], # with (x^4)
 [0.77, 0.25, 0.10], # one (x^5)
 [0.05, 0.80, 0.55]] # step (x^6)
)

query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
 attn_scores_2[i] = torch.dot(x_i, query) # dot là phép vô hướng giữa 2 vector
print(attn_scores_2)

# Kết quả: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Hình minh họa

<img src="https://images.viblo.asia/e429042d-9beb-428d-845f-32f41eb03c45.png" width="800px">

### Chuẩn hóa

Việc chuẩn hóa **Attention Scores** là một bước quan trọng trong cơ chế Self-Attention

Tại sao lại cần chuẩn hóa **attention scores** ?
- Không có ý nghĩa xác suất: Các giá trị Attention Scores không được chuẩn hóa không có ý nghĩa xác suất, trong khi các trọng số attention cần biểu diễn mức độ "tập trung" của mỗi phần tử vào các phần tử khác.
- Không ổn định: Các giá trị **attention scores** có thể là những số rất lớn hoặc rất bé. Nếu dùng các giá trị này làm trọng số có thể xảy ra tình trạng thiên lệch khi các số lớn chi phối và làm cho các giá trị nhỏ trở nên không đáng kể.


<img src="https://images.viblo.asia/81714ea0-9871-4040-aa4d-232efb670954.png" width="800px">

Phương pháp phổ biến để chuẩn hóa được sử dụng là hàm **softmax**, các kết quả sẽ được điều chỉnh lại sao cho tổng của tất cả chúng bằng `1`

$\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$

- $\sigma(z)_i$ là xác suất của lớp i.
- $z_i$ là điểm attention (attention score)
- K là tổng số phần tử

Ví dụ có attention scores như sau: `tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])`

- $\sum_{j=1}^{K} e^{z_j} = e^{0.9544} + e^{1.4950} + e^{1.4754} + e^{0.8434} + e^{0.7070} + e^{1.0865} = 18.7453$

=> Giá trị **0.9544** sau khi chuẩn hoá: $\frac{e^{0.9544}}{18.7453} = 0.1385$





In [13]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention scores:", attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention scores: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


Pytorch cung cấp sẵn hàm softmax để sử dụng

In [14]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


### Tính vector context z

Tính **context vector** bằng cách lấy tổng của các tích **trọng số attention** và **token embeddings**. 

<img src="https://images.viblo.asia/ab263633-b9c3-48e3-be54-1234a7ffeaad.png" width="800px">

In [15]:
query = inputs[1] # journey

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Tổng quát hoá

Ở phần trên, ta chỉ đang tính cho từ thứ 2 trong câu là **journey**.

Để tính toàn bộ giá trị của các vector z, ta thực hiện lần lượt các bước như sau:

- 1. Tính **attention scores** bằng cách lấy ma trận **token embeddings** nhân cho ma trận chuyển vị (transpose) của nó.
- 2. Chuẩn hóa **attention scores** thành **trọng số attention**
- 3. Tính **context vector z** bằng cách lấy  **trọng số attention** nhân cho **token embeddings**
​

**Lưu ý**: Việc tính **attention score** mà chúng ta đã thực hiện bản chất là nhân dòng thứ 2 của ma trận `inputs` lần lượt với các cột của ma trận `inputs^T` (ma trận chuyển vị của inputs).


$$ inputs = 
\begin{bmatrix}
0.43 & 0.15 & 0.89 \\
0.55 & 0.87 & 0.66 \\
0.57 & 0.85 & 0.64 \\
0.22 & 0.58 & 0.33 \\
0.77 & 0.25 & 0.10 \\
0.05 & 0.80 & 0.55
\end{bmatrix},
inputs^T = 
\begin{bmatrix}
0.43 & 0.55 & 0.57 & 0.22 & 0.77 & 0.05 \\
0.15 & 0.87 & 0.85 & 0.58 & 0.25 & 0.80 \\
0.89 & 0.66 & 0.64 & 0.33 & 0.10 & 0.55
\end{bmatrix}$$

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- Bước 1: Ma trận 6x3 nhân với ma trận 3x6 cho ra ma trận có kích thước 6x6

In [16]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- Ngắn gọn hơn, ta có

In [17]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- Bước 2: Chuẩn hoá với Softmax

In [18]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- Kiểm tra nhanh xem các giá trị ở dòng thứ 2 có tổng bằng 1 không ?

In [19]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- Bước 3: Tính vector z

In [20]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- $z^{(2)} = [0.4419, 0.6515, 0.5683]$ cũng y chang kết quả tính ở phần trên

In [15]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 Self-Attention với trọng số có thể huấn luyện được (self-attention with trainable weights)

- Với trường hợp self-attention đơn giản, các trọng số được tính toán theo một công thức toán học nhất định => không thể điều chỉnh hay thay đổi.
- Trọng số có thể huấn luyện được là trọng số có thể thay đổi, tối ưu hoá.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="800px">

### 3.4.1 Tính attention weights từng bước một

- Ta có **$W_Q$, $W_K$, $W_V$** lần lượt là ba ma trận.
- 3 ma trận sẽ được sử dụng trong quá trình tính toán trọng số attention.
- Thông số trong các ma trận có thể được điều chỉnh qua quá trình huấn luyện.

- Gọi q, k, v lần lượt là các vector query, key, value

  - Query vector: $q^{(i)} = x^{(i)}\,W_q $
  - Key vector: $k^{(i)} = x^{(i)}\,W_k $
  - Value vector: $v^{(i)} = x^{(i)}\,W_v $


- Kích thước vector $x$ và vector $q$ có thể giống hoặc khác nhau, tùy thuộc vào thiết kế và cách triển khai cụ thể của mô hình.
- Trong các mô hình GPT, kích thước đầu vào và đầu ra thường giống nhau, nhưng để minh họa và dễ theo dõi các phép tính, chúng ta chọn kích thước đầu vào và đầu ra khác nhau ở đây:

<img src="https://images.viblo.asia/c2255234-587d-4f21-b158-dc9de24514d6.png" width="800px">

In [21]:
x_2 = inputs[1] # Từ journey
d_in = inputs.shape[1] # kích thước đầu vào d=3
d_out = 2 # kích thước đầu ra d=2

In [22]:
torch.manual_seed(123)

# requires_grad=False là cài đặt để các ma trận sẽ không cập nhật trong quá trình huấn luyện
# Giữ cố định để chúng ta dễ theo dõi kết quả
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- Tính các vector query, key và value:

In [23]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [24]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- Bước tiếp theo, **Bước 2**: Tính attention score

$ω_{2i} = q^{(2)} \times k^{(i)}$

<img src="https://images.viblo.asia/b9318798-52d2-493f-97ad-67b4af02340d.png" width="800px">

In [25]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22) # omega_22

tensor(1.8524)


- Tính cả 6 giá trị attention score

In [26]:
attn_scores_2 = query_2 @ keys.T # chuyển vị của keys
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


- Tiếp theo, **Bước 3**, tính **attention weights** bằng cách chuẩn hoá **attention scores**.
- Softmax trong trường hợp này là lấy **attention scores** chia cho  $\sqrt{d_k}$ Với $d_k$ là kích thước vector keys

In [27]:
d_k = keys.shape[1]
print("d_k:", d_k)
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

d_k: 2
tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://images.viblo.asia/69eff0fc-a3ff-42a2-8453-d86d09f3ebb2.png" width="800px">

- Tại **Bước 4**, Tính context vector của từ `journey`:

 $z^{2} = α_{(2)} \times v$

In [28]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 Công thức tổng quát

Sau khi đã tính được context vector **$z_2$**.Chúng ta cùng xem các bước tổng quát để tính cả **context vector z**

>Bước 1.  Với **X** là vector input embeddings và $b_q, b_k, b_v$ là các hệ số tự do (bias)
>
>$Q = X \cdot W_Q + b_q$, 
>
>$K = X \cdot W_K + b_k$, 
>
>$V = X \cdot W_V + b_v$
>
> Bước 2. $Attention\ Scores=Q⋅K^{T}$
> 
> Bước 3: $Attention\ Weights = softmax(\frac{QK^T}{\sqrt{d_k}})$ với $d_k$ là số chiều của k (ví dụ trên thì k = 2)
> 
>  Bước 4: $z = Attention Weights⋅V$

In [29]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="800px">

- Chúng ta có sử dụng `Linear` của **PyTorch** thay thế cho `Parameter`

Một số lợi thế lớn khác của việc sử dụng `nn.Linear` so với `nn.Parameter(torch.rand(...)`:
- Có hỗ trợ bias (không dùng thì cho bằng False).
- `nn.Linear` sử dụng phương pháp khởi tạo trọng số được tối ưu hóa giúp mô hình đạt kết quả huấn luyện nhanh hơn. Trong khi đó, `torch.rand(...)` chỉ tạo ngẫu nhiên trong khoảng [0, 1]

In [30]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## 3.Causal attention

**Causal Attention** (hay còn gọi là Masked Attention) là một kỹ thuật được nhằm che dấu các thông tin phía sau khiến mô hình chỉ nắm được các dữ liệu ở phía trước.

Nói đơn giản hơn, điều này đảm bảo rằng việc dự đoán từ tiếp theo chỉ nên phụ thuộc vào các từ đứng trước nó.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="800px">

### 3.5.1 Triển khai Causal attention

- Chuẩn hoá **attention scores** để thu được **attention weights**
- Áp dụng Causal attention thu được **Masked attention scores**
- Chuẩn hoá một lần nữa để thu được **Masked attention weights**

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="800px">

**Bước 1**: Chuẩn hoá attention_scores

In [31]:
# attention_scores => attention_weights
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


**Bước 2**: Áp dụng casual attention

In [32]:
# Tạo 1 ma trận tam giác dưới có kích thước bằng ma trận attention_weights ở trên
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- Nhân 2 ma trận `mask_simple` và `attention_weights` với nhau

In [33]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


**Bước 3**: Chuẩn hoá mask_simple

In [34]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


Cách trên phải áp dụng softmax tận 2 lần, ta cùng đến với cách thứ 2 ngắn gọn hơn

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

- Bước 1: Thay thay các giá trị nằm trên đường chéo chính của ma trận **attention scores** thành `-inf`
- Bước 2: Chuẩn hóa với softmax cho ra **trọng số attention**

In [35]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


So với cách 1, cách này sử dụng ít bước hơn. Khác biệt cơ bản giữa 2 cách là việc thay thế các giá trị trên đường chéo chính thành `-inf`.

Với giá trị `-inf`, $-inf = -\infty \implies e^{-\infty} = 0 \rightarrow \text{xác suất tương ứng} = 0$


In [36]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 Dropout

- Ngoài việc che giấu thông tin bằng **casual attention**. Trong thực tế, người ta còn áp dụng thêm kỹ thuật **Dropout** nhằm loại bỏ ngẫu nhiên một phần tham số của mô hình. 

- Cần lưu ý rằng dropout chỉ được sử dụng trong quá trình huấn luyện.

Hình dưới đây minh họa kỹ thuật **dropout** với tỷ lệ 50%.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="800px">

- Khi áp dụng dropout, các giá trị không bị loại bỏ sẽ được bù lại cách nhân với một hệ số có giá trị `1 / (1 - dropout_rate)`

In [37]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout 50%
example = torch.ones(6, 6) # Ma trận đơn vị 6x6 với các phần tử là 1
print(example)
print(dropout(example))

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [38]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


### 3.5.3 Tổng hợp lại với causal self-attention class

Viết class `CausalAttention`

In [39]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [40]:
# Triển khai causal attention, sau đó áp dụng dropout
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="800px">

## 3.6 Từ single-head attention đến multi-head attention

Sự khác biệt cơ bản giữa **single-head attention** và **multi-head attention** là ở số lượng ma trận **$W_Q$, $W_K$, $W_V$**

### 3.6.1 Multi-head bằng cách xếp chồng nhiều lớp single-head attention

- Dưới đây là hình minh họa cho cấu trúc 2 **single-head attention**

<img src="https://images.viblo.asia/950e0003-9047-435c-b239-c6edc30ada5d.png" width="800px">

- Chúng ta chỉ cần xếp chồng nhiều single-head attention để tạo thành một multi-head attention:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="800px">


- Mỗi head tính toán song song với nhau

In [41]:
class MultiHeadAttentionWrapper(nn.Module):
    # num_heads: head attention
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1]
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- d_out = 2: kích thước của mỗi vector key, query, value trong mỗi đầu attention (attention head).
- Số attention head = 2

=> Mỗi head sẽ tạo ra một context vector có kích thước `d_out = 2`. Vì có 2 head nên vector Z cuối cùng thu được có 2x2 = **4 chiều**


### 3.6.2 Triển khai multi-head attention with weight splits


- Thay vì nhiều $W_{query}, W_{key}, W_{value}$ như ở trên, chúng ta tạo ra các 1 bộ ma trận duy nhất rồi sau đó chia chúng thành các ma trận riêng lẻ cho từng head attention.


Hình dưới minh hoạ sự khác nhau giữa việc có **weight splits** và không **weight splits**

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="800px">

Ưu điểm của việc **weight splits**
- Tối ưu hóa tốt hơn

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        # Đảm bảo d_out chia hết cho số lượng heads
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Kích thước cho mỗi head

        # Các ma trận W
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # Lớp để gộp kết quả từ các head lại
        self.out_proj = nn.Linear(d_out, d_out)

        self.dropout = nn.Dropout(dropout)

        # causal mask
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape  # batch size, số token, kích thước đầu vào

        # Biến đổi input thành các tensor query, key, value
        keys = self.W_key(x)      # Kích thước (shape): (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # Chia nhỏ thành các head: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        print("Split Keys: ", keys)

        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Đưa chiều num_heads lên trước: -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        print("Transpose Keys: ", keys)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Tính attention scores bằng cách nhân Q với K^T
        attn_scores = queries @ keys.transpose(2, 3)  # (b, num_heads, num_tokens, num_tokens)

        # Áp dụng causal attention
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)  # Gán -inf vào các vị trí bị che

        # Chuẩn hóa attention scores bằng softmax
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # Áp dụng dropout
        attn_weights = self.dropout(attn_weights)

        # Tính toán context vector: (b, num_heads, num_tokens, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)  # Đổi lại về (b, num_tokens, num_heads, head_dim)

        # Gộp các head lại: (b, num_tokens, num_heads * head_dim = d_out)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        # Dự đoán đầu ra cuối cùng (có thể học được)
        context_vec = self.out_proj(context_vec)

        return context_vec


Ví dụ cụ thể  về cách chia và biến đổi các head

In [63]:
x = [
 [[1,2,3,4,5,6],   [7,8,9,10,11,12],   [13,14,15,16,17,18]],
 [[19,20,21,22,23,24],   [25,26,27,28,29,30],   [31,32,33,34,35,36]]
]

# Chuyển sang Tensor
x = torch.tensor(x, dtype=torch.float32)

# Batch 0 có 3 token
# Batch 1 cũng có 3 token
# Mỗi token là 1 vector 6 chiều.

b, num_tokens, d_in = x.shape # 2, 3, 6
num_heads = 2
d_out = 6
head_dim = d_out // num_heads # 3
dropout = 0.1

In [64]:
multihead_attention = MultiHeadAttention(d_in, d_out, context_length, dropout, num_heads)

In [65]:
output = multihead_attention(x)

Split Keys:  tensor([[[[ -1.2433,  -4.1203,   0.3265],
          [ -0.9697,   5.6101,  -5.8388]],

         [[ -4.0821,  -7.7870,   0.9813],
          [ -2.2691,  14.9648, -15.1872]],

         [[ -6.9209, -11.4537,   1.6360],
          [ -3.5685,  24.3196, -24.5356]]],


        [[[ -9.7597, -15.1204,   2.2908],
          [ -4.8678,  33.6743, -33.8840]],

         [[-12.5985, -18.7871,   2.9455],
          [ -6.1672,  43.0290, -43.2324]],

         [[-15.4373, -22.4538,   3.6003],
          [ -7.4666,  52.3837, -52.5808]]]], grad_fn=<ViewBackward0>)
Transpose Keys:  tensor([[[[ -1.2433,  -4.1203,   0.3265],
          [ -4.0821,  -7.7870,   0.9813],
          [ -6.9209, -11.4537,   1.6360]],

         [[ -0.9697,   5.6101,  -5.8388],
          [ -2.2691,  14.9648, -15.1872],
          [ -3.5685,  24.3196, -24.5356]]],


        [[[ -9.7597, -15.1204,   2.2908],
          [-12.5985, -18.7871,   2.9455],
          [-15.4373, -22.4538,   3.6003]],

         [[ -4.8678,  33.6743, -33.8840]

Split Keys:
```python
tensor([
      [ # batch 0
        [[ -1.4421,   0.5373,   0.5513],[ -1.6091,  -0.8023,   0.4414]], # token 0 -> head 0: [-1.4421,   0.5373,   0.5513], head 1: [1.6091,  -0.8023,   0.4414]
        [[ -3.8594,   3.2438,   1.5168], [ -5.4921,  -0.9958,   3.2902]],# token1
        [[ -6.2767,   5.9502,   2.4822],[ -9.3751,  -1.1893,   6.1390]]  # token2
      ],
      [ # batch 1
        [[ -8.6940,   8.6567,   3.4477], [-13.2580,  -1.3828,   8.9877]], # token3
        [[-11.1113,  11.3632,   4.4131], [-17.1410,  -1.5763,  11.8365]], # token4
        [[-13.5286,  14.0696,   5.3785], [-21.0240,  -1.7698,  14.6853]]  # token5
      ]
    ], grad_fn=<ViewBackward0>)
```

Transpose Keys: Biến đổi lại để mỗi head xử lý các tokens của riêng nó.
```python
tensor([[
        # Các token do head0 xử lý
        [
            [ -1.4421,   0.5373,   0.5513],
            [ -3.8594,   3.2438,   1.5168],
            [ -6.2767,   5.9502,   2.4822]
        ],

        # Các token do head1 xử lý
        [
            [ -1.6091,  -0.8023,   0.4414],
            [ -5.4921,  -0.9958,   3.2902],
            [ -9.3751,  -1.1893,   6.1390]
        ]],
```

In [ ]:
# In kết quả cuối cùng
print("Output shape:", output.shape)
print("Output:\n", output)

Output shape: torch.Size([2, 3, 6])
Output:
 tensor([[[  0.1784,   0.0321,   1.4047,  -1.5837,   1.2440,  -1.6398],
         [  0.0717,   0.1074,   1.4796,  -1.8049,   1.2127,  -1.7823],
         [  0.1723,   0.0364,   1.4089,  -1.5963,   1.2422,  -1.6479]],

        [[  1.0399,   5.8948,   4.3728, -16.1517,   1.6774,  -8.0966],
         [  1.0399,   5.8948,   4.3728, -16.1518,   1.6774,  -8.0966],
         [  1.0399,   5.8948,   4.3728, -16.1517,   1.6774,  -8.0966]]],
       grad_fn=<ViewBackward0>)
